In [1]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon

# Load the Shapefile
shapefile_path = '../Lower layer Super Output Areas (December 2001) Boundaries EW BFC/LSOA_Dec_2001_EW_BFC_2022_-3090597439272179543/LSOA_2001_EW_BFC_V2.shp'
lsoa_gdf = gpd.read_file(shapefile_path)

# Ensure the GeoDataFrame has the correct coordinate reference system (CRS)
lsoa_gdf = lsoa_gdf.to_crs(epsg=4326)


In [2]:
lsoa_gdf.head()

,LSOA01CD,LSOA01NM,LSOA01NMW,GlobalID,geometry
0,E01000001,City of London 001A,City of London 001A,cc30fabd-068b-4308-8f6b-f7c3b5a70bd9,"POLYGON ((-0.09667 51.52027, -0.09666 51.52025..."
1,E01000002,City of London 001B,City of London 001B,14c8ee1b-d6f0-4afe-a05c-14a69530e38e,"POLYGON ((-0.08969 51.52069, -0.08973 51.52057..."
2,E01000003,City of London 001C,City of London 001C,c6b8af00-b5da-4582-8019-3ccc9c294e50,"POLYGON ((-0.09653 51.52295, -0.09647 51.52282..."
3,E01000004,City of London 001D,City of London 001D,7f890d5b-77d9-49b7-b355-605c03f3e2e5,"POLYGON ((-0.07891 51.52041, -0.07910 51.51971..."
4,E01000005,City of London 001E,City of London 001E,c0158609-c7de-4fed-92bd-611ed8284dce,"POLYGON ((-0.07571 51.51575, -0.07542 51.51555..."


In [3]:
import pandas as pd

participants = pd.read_csv('../LSOA_participants_unique.csv', index_col=None)
LSOA_use = list(participants['LSOA_code'].unique())
participants.head()

,Unnamed: 0,LSOA_code,id,area,LSOA11NM,TCITY15CD,TCITY15NM,FID
0,1,E01000001,3930876,London,City of London 001A,J01000055,London,1.0
1,2,E01000001,4650325,London,City of London 001A,J01000055,London,1.0
2,3,E01000001,3118216,London,City of London 001A,J01000055,London,1.0
3,4,E01000001,2126547,London,City of London 001A,J01000055,London,1.0
4,5,E01000001,5323908,London,City of London 001A,J01000055,London,1.0


In [4]:
LSOA_london = list(participants[participants['TCITY15NM']=="London"]["LSOA_code"].unique())
len(LSOA_london)

2936

In [5]:
london = lsoa_gdf[lsoa_gdf['LSOA01CD'].isin(LSOA_london)]

In [6]:
london.shape

(2936, 5)

In [7]:
london['area'] = london['geometry'].to_crs(epsg=3857).area
london = london[['LSOA01CD','LSOA01NM','geometry','area']]

c:\Users\yueli\miniconda3\envs\myenv\lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [8]:
london.head()

,LSOA01CD,LSOA01NM,geometry,area
0,E01000001,City of London 001A,"POLYGON ((-0.09667 51.52027, -0.09666 51.52025...",3.349810e+05
1,E01000002,City of London 001B,"POLYGON ((-0.08969 51.52069, -0.08973 51.52057...",5.891986e+05
2,E01000003,City of London 001C,"POLYGON ((-0.09653 51.52295, -0.09647 51.52282...",1.523513e+05
3,E01000004,City of London 001D,"POLYGON ((-0.07891 51.52041, -0.07910 51.51971...",5.908310e+06
4,E01000005,City of London 001E,"POLYGON ((-0.07571 51.51575, -0.07542 51.51555...",4.889038e+05


In [7]:
# Define the function to get historical OSM data from ohsome API
def get_total_area(geometry, date="2019-01-01", filter_='type:way'):
    base_url = "https://api.ohsome.org/v1/elements/geometry"
    
    data = {
        "bpolys": geometry,
        "time": date,
        "filter": filter_
    }
    
    response = requests.post(base_url, data=data)
    
    if response.status_code == 200:
        features = response.json()['features']
        coordinate_list = []

        for feature in features:
            type = feature['geometry']['type']
            if type == 'Polygon':
                coordinates = feature['geometry']['coordinates']
                coordinate_list.append(coordinates[0])
            elif type == 'MultiPolygon':
                coordinates = feature['geometry']['coordinates']
                for c in coordinates:
                    coordinate_list.append(c[0])
            else:  # mutipolygon suspect
                print('Unkown SHAPE WARNING')
                print(type)
    
        return calculate_area(coordinate_list)
    
    else:
        print(f"Error fetching data: {response.status_code}")
        print(response.json())
        return None
    
from shapely.ops import unary_union

def calculate_area(coordinates):
    """
    Input: a list of coordinate lists like [[[1,2],[3,4],[5,6],[1,2]],[[7,8],[9,10],[11,12],[7,8]]]
    Output: the area of the union of all polygons in the input list
    """
    polygons = []
    
    for coord_list in coordinates:
        # Convert each sublist into a tuple to create a Polygon
        polygon = Polygon([tuple(coord) for coord in coord_list])
        
        # Check if the polygon is valid, if not, fix it using buffer(0)
        if not polygon.is_valid:
            polygon = polygon.buffer(0)
        
        polygons.append(polygon)
    
    # Create a GeoDataFrame with all polygons
    gdf = gpd.GeoDataFrame(index=range(len(polygons)), crs="EPSG:4326", geometry=polygons)
    
    # Convert to a projected coordinate system for accurate area calculation
    gdf = gdf.to_crs(epsg=3857)
    
    # Compute the union of all polygons
    union_polygon = unary_union(gdf['geometry'])
    
    # Calculate the area of the union polygon
    total_area = union_polygon.area
    
    return total_area

def polygon_to_coord_list(polygon):
    if isinstance(polygon, Polygon):
        coords = list(polygon.exterior.coords)
        coord_list = ",".join([f"{x},{y}" for x, y in coords])
        return [coord_list]
    elif isinstance(polygon, MultiPolygon):
        coord_list = []
        for poly in polygon.geoms:
            coords = list(poly.exterior.coords)
            coord_list.append(",".join([f"{x},{y}" for x, y in coords]))
        return coord_list
    else:
        raise TypeError("Unsupported geometry type")

In [2]:
filters = [
# green space
"geometry:polygon and (landuse in (allotments, farmland, farmyard, paddy, forest, meadow, animal_keeping, orchard, vineyard, plant-nursery, grass, village_green) or leisure in (garden, nature_reserve, park, pitch) or natural in (fell, grassland, health, scrub, tundra, wood))",
]
var_names = ['green_space']

In [11]:
# initiate an empty columns for result recording
import pandas as pd

var_names = ['green_space']
for var in var_names:
    london[var] = [pd.NA] * london.shape[0]


In [12]:
london.head()

,LSOA01CD,LSOA01NM,geometry,area,green_space
0,E01000001,City of London 001A,"POLYGON ((-0.09667 51.52027, -0.09666 51.52025...",3.349810e+05,<NA>
1,E01000002,City of London 001B,"POLYGON ((-0.08969 51.52069, -0.08973 51.52057...",5.891986e+05,<NA>
2,E01000003,City of London 001C,"POLYGON ((-0.09653 51.52295, -0.09647 51.52282...",1.523513e+05,<NA>
3,E01000004,City of London 001D,"POLYGON ((-0.07891 51.52041, -0.07910 51.51971...",5.908310e+06,<NA>
4,E01000005,City of London 001E,"POLYGON ((-0.07571 51.51575, -0.07542 51.51555...",4.889038e+05,<NA>


In [13]:
# for y in ['2019','2014','2012','2010']:
#     london.to_csv('london_area_'+y+'.csv')

In [9]:
from tqdm import tqdm
from shapely import wkt
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon

for y in ['2019','2014','2012','2010']:
    london = pd.read_csv('london_area_'+y+'.csv')
    london['geometry'] = london['geometry'].apply(wkt.loads)
    for i, row in tqdm(london.iterrows(), total=london.shape[0]):
        if row.isna().any() or (-1 in row.values):
            geometry_list = polygon_to_coord_list(row['geometry'])
            for j in range(len(filters)):
                if pd.isna(row[var_names[j]]) or (row[var_names[j]] == -1):
                    try:
                        total_area = 0
                        for geometry in geometry_list:
                            area = get_total_area(geometry, y+'-01-01', filters[j])
                            total_area += area
                        london.at[i, var_names[j]] = total_area
                    except Exception as e:
                        print(f"Error fetching POI data for LSOA {row['LSOA01CD']}: {e}")
                        london.at[i, var_names[j]] = -1

            # Save the updated DataFrame to the CSV file
            if i % 10 == 0:
                london.to_csv('london_area_'+y+'.csv', index=False)
    london.to_csv('london_area_'+y+'.csv', index=False)

 24%|██▍       | 702/2936 [01:05<03:30, 10.64it/s] 


KeyboardInterrupt: 

In [16]:
london.head()

,Unnamed: 0,LSOA01CD,LSOA01NM,geometry,area,commercial_industrial
0,0,E01000001,City of London 001A,POLYGON ((-0.096673331375757 51.52027362452771...,3.349810e+05,44313.804456
1,1,E01000002,City of London 001B,POLYGON ((-0.0896947088045908 51.5206871841854...,5.891986e+05,35627.812011
2,2,E01000003,City of London 001C,POLYGON ((-0.0965307593953206 51.5229489233266...,1.523513e+05,24886.962824
3,3,E01000004,City of London 001D,POLYGON ((-0.0789101868202144 51.5204064571767...,5.908310e+06,172488.237098
4,4,E01000005,City of London 001E,POLYGON ((-0.0757107886483168 51.5157497490281...,4.889038e+05,6601.437467


In [20]:
stats = []
for y in ['2010','2012','2014','2019']:
    london = pd.read_csv('london_streetnetwork_new_'+y+'.csv')
    year_stats = []
    for v in var_names:
        year_stats.append(london[v].sum())
    stats.append(year_stats)
stats = pd.DataFrame(stats)
stats.columns = var_names
    

In [21]:
stats.index = ['2010','2012','2014','2019']

In [22]:
stats

# results should delete:
# amenity_biergarten, shop_brewing_supplies, shop_cannabis

,cycling_cycleway,cycling_highway,walking_sidewalk,walking_highway,walking_footway
2010,122672.638737,235059.695756,0.000000e+00,1.458632e+06,222.041542
2012,215957.993623,287091.112709,2.314453e+03,1.868238e+06,13613.436030
2014,290649.677063,365215.962422,1.291878e+06,2.376629e+06,58182.469202
2019,284174.776649,644677.810531,1.819262e+06,2.893637e+06,158894.659990


In [18]:
# backup function

# import json
# from shapely.geometry import Polygon, MultiPolygon

# def polygon_to_geojson_feature_collection(polygon, region_id='Region'):
#     features = []

#     def format_coordinates(coords):
#         return [[list(coord) for coord in coords]]

#     if isinstance(polygon, Polygon):
#         coordinates = format_coordinates(polygon.exterior.coords)
#         feature = {
#             "type": "Feature",
#             "properties": {"id": f"{region_id} 1"},
#             "geometry": {
#                 "type": "Polygon",
#                 "coordinates": coordinates
#             }
#         }
#         features.append(feature)

#     elif isinstance(polygon, MultiPolygon):
#         for idx, poly in enumerate(polygon.geoms):
#             coordinates = format_coordinates(poly.exterior.coords)
#             feature = {
#                 "type": "Feature",
#                 "properties": {"id": f"{region_id} {idx + 1}"},
#                 "geometry": {
#                     "type": "Polygon",
#                     "coordinates": coordinates
#                 }
#             }
#             features.append(feature)
#     else:
#         raise TypeError("Unsupported geometry type")

#     feature_collection = {
#         "type": "FeatureCollection",
#         "features": features
#     }
#     return feature_collection


{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'id': 'Region 1'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-1.5704498471484643, 54.923952175382496], [-1.5704497077420054, 54.92395207963483], [-1.5701740091293421, 54.92406343507078], [-1.570067292381269, 54.92410507449073], [-1.5699574872283233, 54.92414791875118], [-1.5698547810752077, 54.92418799326264], [-1.569747889753728, 54.924229700081426], [-1.5697289359590474, 54.92423709530048], [-1.5696262192179127, 54.92428016742882], [-1.569531771414466, 54.92431977124733], [-1.5694002422589792, 54.924374924008276], [-1.5692680481964985, 54.92443035554523], [-1.5690005287472784, 54.92455891469998], [-1.568879389802446, 54.9246171303187], [-1.5688792645069296, 54.92461702832883], [-1.5688746180357085, 54.92461321871992], [-1.5688275856568914, 54.924574662304586], [-1.5682360406280935, 54.92408970825292], [-1.5679862770907975, 54.92390418146634], [-1.567864526165285, 54.92381374343603], [-1.567676365597

In [16]:
london

,Unnamed: 0,LSOA01CD,LSOA01NM,geometry,area,cycling_cycleway,cycling_highway,walking_sidewalk,walking_highway,walking_footway
0,0,E01000001,City of London 001A,POLYGON ((-0.096673331375757 51.52027362452772...,3.349810e+05,391.057897,0.000000,0.0,1071.220039,0.0
1,1,E01000002,City of London 001B,POLYGON ((-0.0896947088045908 51.5206871841854...,5.891986e+05,254.609331,0.000000,0.0,2195.263472,0.0
2,2,E01000003,City of London 001C,POLYGON ((-0.0965307593953206 51.5229489233266...,1.523513e+05,144.557958,0.000000,0.0,366.989688,0.0
3,3,E01000004,City of London 001D,POLYGON ((-0.0789101868202144 51.5204064571767...,5.908310e+06,870.006297,403.399653,0.0,17467.349464,0.0
4,4,E01000005,City of London 001E,POLYGON ((-0.0757107886483168 51.5157497490281...,4.889038e+05,7.799653,65.940872,0.0,625.390290,0.0
...,...,...,...,...,...,...,...,...,...,...
2931,23993,E01023994,Ashford 012A,POLYGON ((0.835814912743647 51.163744409642014...,3.175157e+07,0.000000,1024.807132,0.0,1088.426374,0.0
2932,24029,E01024030,Ashford 008D,POLYGON ((0.8470910517428684 51.13284950871105...,2.492635e+06,0.000000,2175.540006,0.0,682.135599,0.0
2933,26850,E01026851,Norwich 008E,MULTIPOLYGON (((1.3038276491026355 52.62655980...,4.039804e+06,135.714291,600.971383,0.0,1776.443954,0.0
2934,30396,E01030397,Epsom and Ewell 006E,POLYGON ((-0.2260138030290731 51.3508820931176...,2.465671e+06,26.649067,0.000000,0.0,1125.835356,0.0


In [19]:
# for y in ['2010','2012','2014','2019']:
#     new = pd.read_csv('london_streetnetwork_new_'+y+'.csv')
#     old = pd.read_csv('london_streetnetwork_'+y+'.csv')
#     if new['LSOA01CD'].equals(old['LSOA01CD']):
#         combined = pd.concat([new[['LSOA01CD','LSOA01NM','geometry','area']],old[['driving']],new[['cycling_cycleway','cycling_highway','walking_sidewalk','walking_highway','walking_footway']]],axis=1)
#         combined.to_csv('london_streetnetwork_new_'+y+'.csv')

In [18]:
combined

,LSOA01CD,LSOA01NM,geometry,area,driving,cycling_cycleway,cycling_highway,walking_sidewalk,walking_highway,walking_footway
0,E01000001,City of London 001A,POLYGON ((-0.096673331375757 51.52027362452772...,3.349810e+05,2746.624124,631.261436,77.558767,2017.149353,2362.960351,61.540343
1,E01000002,City of London 001B,POLYGON ((-0.0896947088045908 51.5206871841854...,5.891986e+05,3852.265548,581.073972,103.156729,1168.368870,5592.564139,219.606739
2,E01000003,City of London 001C,POLYGON ((-0.0965307593953206 51.5229489233266...,1.523513e+05,877.511897,175.611017,12.749535,613.207608,987.987545,0.000000
3,E01000004,City of London 001D,POLYGON ((-0.0789101868202144 51.5204064571767...,5.908310e+06,49041.610707,9573.410392,5095.208918,26400.481176,37881.666230,4390.770068
4,E01000005,City of London 001E,POLYGON ((-0.0757107886483168 51.5157497490281...,4.889038e+05,3222.702365,609.004682,810.170906,2225.463290,2373.983222,578.443965
...,...,...,...,...,...,...,...,...,...,...
2931,E01023994,Ashford 012A,POLYGON ((0.835814912743647 51.163744409642014...,3.175157e+07,32645.783619,0.000000,1409.595396,0.000000,6674.122509,0.000000
2932,E01024030,Ashford 008D,POLYGON ((0.8470910517428684 51.13284950871105...,2.492635e+06,11344.438421,0.000000,2516.697153,99.343065,916.219755,567.704384
2933,E01026851,Norwich 008E,MULTIPOLYGON (((1.3038276491026355 52.62655980...,4.039804e+06,12951.369340,283.352613,294.890856,5604.566943,5859.204433,1005.141651
2934,E01030397,Epsom and Ewell 006E,POLYGON ((-0.2260138030290731 51.3508820931176...,2.465671e+06,8136.713296,29.640900,0.000000,0.000000,1289.607954,0.000000
